# Indeed webscraper

page 1= https://nl.indeed.com/jobs?q=data+analist&start=00  <br>
page 2= https://nl.indeed.com/jobs?q=data+analist&start=10 <br>
page 3= https://nl.indeed.com/jobs?q=data+analist&start=20 <br>


**Getting the link of the job**:

`<table id="resultsBody"> --> <td id="resultsCol"> --> <a (containing href=)>` --> href == link to job

**Getting job description**:
`<div id="jobDescriptionText"> --> <p> of (<ul> --> <li>)` --> paragraphs of text


## Load libraries

In [1]:
library(tidyverse)
library(rvest)
library(XML)
library(stringr)
library(lubridate)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




In [61]:
job_base_url <- 'https://nl.indeed.com'
job_base_search_url <- 'https://nl.indeed.com/jobs?q=data&start='
# url_job_search <- read_html('https://nl.indeed.com/jobs?q=data')

wordlist <- read.table(
    '..\\Capstone\\trimmed_skill_names.txt',
     header = TRUE
     ) %>% .$x

jobs_df <- data.frame(
    Company = character(),
    job_title = character(),
    salary = character(),
    skills = character(),
    days_online = numeric(),
    job_desc = character(),
    job_link = character()
)

for (i in seq.default(0, 10, 10)){

    job_list_url <- paste(job_base_search_url, as.character(i), sep= "")
    job_list_read <- read_html(job_list_url) # lees html in

    job_urls <- job_list_read %>%
        html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
        html_nodes('a') %>% # ga naar a
        html_attr("href")# pak de href

    # loop over job_urls
    for (j in 1:length(job_urls)){

        # when url is longer than 200 chars it is a job url
        # nchar(job_urls[[j]]) > 200
        if (grepl('/pagead/', job_urls[[j]]) | grepl('/clk?', job_urls[[j]]) | grepl('/company/', job_urls[[j]])  ){
            full_job_url <- paste(job_base_url, job_urls[[j]], sep="")
            print(full_job_url)
            # read html
            job_html_read <- read_html(full_job_url)

            # extract company name
            company_name <- job_html_read %>%
                html_nodes(css= 'head > meta:nth-child(15)') %>%
                html_attr('content')

            # extract title
            job_title <- job_html_read %>%
                html_nodes('h1') %>%
                html_text()
            
            time_since_placed_text <- job_html_read %>%
                html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
                html_text()

            # check if string does not contain integer using regex
            if (!grepl("\\d", time_since_placed_text) & !grepl("Vandaag", time_since_placed_text)){
                time_since_placed_text <- job_html_read %>%
                    html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
                    html_text()
            }

            if (grepl("\\d", time_since_placed_text)){
                time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]]
            } else if (grepl("Vandaag", time_since_placed_text)) {
                time_since_placed <- 0
            } else {
            time_since_placed <- NA
            }


            # extract description
            job_description <- job_html_read %>%
                html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
                html_text()
            
            # remove non alpha numeric characters
            job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

            # generate wordvector
            wordvec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

            # extract skills from word vector
            skill_table <- wordvec[wordvec %in% tolower(wordlist)] %>%
                unique()

            # probeer salari op te halen via specifieke salary indicator
            salary_indicator <- job_html_read %>%
                html_nodes(css= '#salaryInfoAndJobType') %>%
                html_text()

            # Als de € niet voorkomt in het bestand probeer dan de salary distribution
            if (length(salary_indicator) < 1){
                salary_indicator <- job_html_read %>%
                    html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
                    html_text()
            }

            if (length(salary_indicator) < 1){
                salary <- NA
            } else if (grepl('€',salary_indicator)) {
                # anders zoek de eerste salaris indicatie
                salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
                salary <- substring(salary_indicator, salary_start_num, salary_start_num + 7) 
            } else {
                salary <- NA
            }
            
            jobs_df[nrow(jobs_df) + 1,] = c(
                company_name,
                job_title,
                salary,
                toString(unique(skill_table)),
                as.numeric(time_since_placed),
                job_description,
                full_job_url)
            
            Sys.sleep(4)
        }
    }
}

[1] "https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dg1aOKV4dKLiKzD6mofKwDD4SbN1IWH_lyY4aNMJ8U1VKP9PN_mQrcSqMdhIm3LEk103q7ZCFeqXwZFt4t8h_i8pvsHXFJFrqqAaGVKJT96uCoY-6st3DKpF0YRBvNwSB6oA99vI3LV-_pvTgsfW-s6NHkkI2s82ExwsUz0OVPDv-ltEUKDbcJXWugbVPBtg3ngaE5ZVctn7a12uZ4zVmsjyxUwOsD61l7QppR7oODKDdAsuChZk1tx1IDVjAYdFeElPhWytNYWOpodCXsjbjqhOXFx3AeblMhllakLgxnKwTPeftIwV03_bucAOJ8kobRdC0n-dKG9PxgtK9k7Lv76iPIy0r42fiLiVaTyRHEZTF0qWdcLrjHCaPygFNNPAPFe36wjs8zA4ylrXUHuUxS4NhCLvRES_JroyGvcYvDT_KSuNLKwykVpGTM_9rmBES-PqkcD9f3RM0X_4OCZJ2ne8uEhGM0MGBKET1pvkr72uFORwHVezRM-APbbu3Vh_Mse-KIphDJdr8m7ZOSZGi8fK6w1uDOsqE65gauYhUOwLmGNYo31sBV-6TLMjUKtXlCZExFKMRePlSDrd2W-ByLMZqWDy2HuXBpmafgQsLGE-cqsR3h8DqWx03znoFQRu7ylwC9qKQdQn9_U9SL1wEQ7ud_VuVN1w_J_JMruA_4puFzAb3WD8zQX9F7astjG6L88QLJc9nOFoN2qBQo2BaB1CdIa5ZAZZevJ2DxhCo2bA==&p=0&fvj=0&vjs=3"
[1] "https://nl.indeed.com/company/World-of-Content/jobs/Data-Scientist-1c0ef28d2ec3b4d2?fccid=d1d45cdb6b1ec5e9&vjs=3"
[1] "https://nl.indeed.com/company/Data-Consulting-Group/

Warning message in matrix(value, n, p):
"data length [8] is not a sub-multiple or multiple of the number of columns [7]"


[1] "https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BQVpz0dABwcJxPL6ZF89l8aMZ4ujW6gW5aOiSkCGD8kHrUJtmUIy1MbgNJvpw90S7uI6Xj3oLI9JITeDQiZXXB5-hUHsD1Civ70sUlnaIMXZKITdj6UozcgDNfipxKcuFCn0C4iEi5Mgvjd7c-nbLc-r1q7WfAx_vA9rng-LIPF7v1YnevH-kAVmrDQ2gvpP4_5WGqet7R3ENDiud5WGwH3-tPHSh7J_w1sWyBlekjPDICJ8jtrbX2M0KM8BWEEcpdnfYJoFIQ0qBZ8AEDwLnH_W2vFuPAmkSnNrS0KUQdvMP_QEsp28DOs3ZJ-Qp85ZqLatWjHWLhAC-bXaDr1QRBVLNzTKQhBXjb6akhRAYMCojbp0V8N3bJu9DGxDavBTG6rrmNhfR5hpQBJw8RP0_G_CoY56Iv0EhO1aX3d0ekq26lr4ytvk-3eRMuoHSSoUdROPhx1V8QTOjlHQPmvdz2rjZblAlfYqh3dqZOHZC4zGq9bRnpMTeFkNAsQUVGyRbaNe0LMlAiuunrMrPqVcQvO6KbzuHxz7tfI6y8IK1sXDVqmP9nJpnsF4y-7P-_qWptRgvMJaC9oeFHhcwo92hZIB7rCpV5QMCzZtlIxL9_EQ==&p=8&fvj=1&vjs=3"
[1] "https://nl.indeed.com/company/Dilaco/jobs/Data-Scientist-b708817cc9516df8?fccid=2563c50176a9aa47&vjs=3"
[1] "https://nl.indeed.com/rc/clk?jk=3404847a23915586&fccid=f255d6d751297923&vjs=3"


Warning message in matrix(value, n, p):
"data length [8] is not a sub-multiple or multiple of the number of columns [7]"


[1] "https://nl.indeed.com/rc/clk?jk=10937df4f2b852c3&fccid=12ba83711d8ef09f&vjs=3"
[1] "https://nl.indeed.com/rc/clk?jk=c387575393cf8753&fccid=113bb7f6362ec380&vjs=3"
[1] "https://nl.indeed.com/rc/clk?jk=5c6eb09ac328c05c&fccid=2b4a63ff7492f90e&vjs=3"
[1] "https://nl.indeed.com/rc/clk?jk=af08fdba5586cb4a&fccid=f255d6d751297923&vjs=3"


Warning message in matrix(value, n, p):
"data length [8] is not a sub-multiple or multiple of the number of columns [7]"


In [62]:
today() - as.numeric(jobs_df$days_online)

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"


[1] "2022-02-17" "2022-03-04" "2022-02-17" "2022-02-22" "2022-02-21"
 [6] "2022-02-02" "2022-02-25" "2022-02-02" "2022-02-02" "2022-03-03"
[11] "2022-02-21" "2022-02-23" "2022-02-02" "2022-02-14" "2022-02-03"
[16] "2022-02-21" "2022-02-03" "2022-02-23" "2022-02-02" "2022-02-14"
[21] "2022-02-03" "2022-02-02" NA           "2022-02-02" "2022-02-14"
[26] NA           "2022-02-07" "2022-02-23" "2022-02-05" NA

In [70]:
file_n<-sprintf("% s scraped indeed data.csv", today())
write.csv(jobs_df, file_n)


## getting indeed search page information
base url= https://nl.indeed.com/jobs?q=data+analist

In [ ]:
url_job_search <- read_html('https://nl.indeed.com/jobs?q=data') # lees html in

job_urls <- url_job_search %>%
  # html_nodes('body')%>%
  html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
  html_nodes('a') %>% # ga naar a
  html_attr("href")# pak de href

job_urls[[1]]

## Get Job info

In [ ]:
job_base_url <- 'https://nl.indeed.com'
full_job_url <- paste(job_base_url, job_urls[[1]], sep="")
full_job_url

In [ ]:
job_description <- read_html(full_job_url) %>%
    html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
    html_text()

## Extract Salary

In [ ]:
test_url = 'https://nl.indeed.com/viewjob?jk=33d4be78835757d1&tk=1ft2v35nf3b5m000&from=serp&vjs=3&advn=4610837443530944&adid=371890778&ad=-6NYlbfkN0A_cWh5ufxJCEatf9FOtgdwl-LajcKrnY3NwdklpeSDnjaH9EsBM6ngV0apa0_Kx2a5GKCW4uS0O7MdMAqsdKUS4q0L_C77bzQCZHbh2VakwHOoBSx_gXNPBJOeqxq7IUUc4s9CBf_lt5ktVCCKOckMPI8DQ-kKHnVfI97NdL-7of8Oo7OCSaCBZL8Z41xkIGi-jjU-emgJ8f1utRtU276rmh-6Rww5rOQKaWr9wwgGy6sYrcY4MEIcYBkbm5BychuYGL0AFbd7Vy65yEMtdbRprQHT9KeUwpjRNpt6FLJMpPGJcxEUzshqpQrMGS0B_CRCLlmxqTWqBEPfpy3Y7aujfdqxmlVIkXqhghqlEzA3Ag==&sjdu=0ZFwD5rbjMRcHz87Kzx_g4iztdTb_2bOBnSpyL0eJqrnoH0zhNHGeUS0d52A-25mtSPp3ZGAbkfQnEhseHJ_YqiTRf1RGyCMrfm3GKjpLJXmRb1As35_i94VtIfO7AZtgI0QXqDAYweserYyqDu1JUW2c6S_FT2kfNZmCatNflk'

salary_indicator <- read_html(test_url) %>%
    html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
    html_text()

salary_indicator

# /html/head/meta[12]
# /html/head/meta[13]
#html.js-focus-visible head meta
#/html/head/meta[13]
#head > meta:nth-child(15)
# head > meta:nth-child(14)

In [ ]:
test_url = 'https://nl.indeed.com/viewjob?jk=aee5f00cab6957ed&q=SAS&tk=1ft62l0v1i9b7802&from=hpd&advn=1272578650180431&adid=367702106&ad=-6NYlbfkN0Bxu6CKaaZeHzJ6IiDpuxwlJ-Jh_LThrAM6mK1WPabT4wOrE7tVDSerQupuoNaldqveRRnZCn-I_tIp5wyi9ean77lpFwBWstwxWV6ehvvWPTacCGfEVwci_XNF1I6UiLnkRlxMaZxMJ1A0xzl8P6ndKdEQCOLPjfz7mv7LBNSdXAwbuEBBwSRXujtr445DebYvoFEty_c9Vrt6wdaxxEfJ2ve64b_DHepatrE8CzLXyC-Qy_cPPLSSFsU32boQAAWRX79YQJhOtj4yzea5N11zenTDua9hFlmQN427rfZLBZHFobUmR28-AIP-Hld2yuVYYFFeREW8oSPbaicTI_TD9R9NNequqrTC88g6KiCW9F-VM18wDDRihihTtZx9-s4TgWF38xVy56wWpuRxuL3STAAyakfmT9_XxKQGpMRR7Q%3D%3D&pub=4a1b367933fd867b19b072952f68dceb&vjs=3'

salary_indicator <- read_html(test_url) %>%
    html_nodes(css= '#salaryInfoAndJobType') %>%
    html_text()


salary_indicator

In [ ]:
# probeer salaris op 1 van de 2 manieren te bepalen
test_url = 'https://nl.indeed.com/viewjob?jk=aee5f00cab6957ed&q=SAS&tk=1ft62l0v1i9b7802&from=hpd&advn=1272578650180431&adid=367702106&ad=-6NYlbfkN0Bxu6CKaaZeHzJ6IiDpuxwlJ-Jh_LThrAM6mK1WPabT4wOrE7tVDSerQupuoNaldqveRRnZCn-I_tIp5wyi9ean77lpFwBWstwxWV6ehvvWPTacCGfEVwci_XNF1I6UiLnkRlxMaZxMJ1A0xzl8P6ndKdEQCOLPjfz7mv7LBNSdXAwbuEBBwSRXujtr445DebYvoFEty_c9Vrt6wdaxxEfJ2ve64b_DHepatrE8CzLXyC-Qy_cPPLSSFsU32boQAAWRX79YQJhOtj4yzea5N11zenTDua9hFlmQN427rfZLBZHFobUmR28-AIP-Hld2yuVYYFFeREW8oSPbaicTI_TD9R9NNequqrTC88g6KiCW9F-VM18wDDRihihTtZx9-s4TgWF38xVy56wWpuRxuL3STAAyakfmT9_XxKQGpMRR7Q%3D%3D&pub=4a1b367933fd867b19b072952f68dceb&vjs=3'

# probeer eerst op te halen via specifieke salary indicator
salary_indicator <- read_html(test_url) %>%
    html_nodes(css= '#salaryInfoAndJobType') %>%
    html_text()


# Als de € niet voorkomt in het bestand probeer dan de salary distribution
if (!grepl( "€", salary_indicator, fixed = TRUE)){
    salary_indicator <- read_html(test_url) %>%
        html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
        html_text()
}

# Als nu de salary_indicator lengte 0 heeft vul de waarden na met NA
if (!length(salary_indicator)){
    salary <- NA
} else {
    salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
    salary_end_num <- unlist(gregexpr(' ', salary_indicator))[1]

    salary <- substring(salary_indicator, salary_start_num, salary_end_num)  
}

salary_indicator
salary


## remove characters from vector

In [ ]:
job_description

job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

job_description_cleaned

word_vec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

word_vec

In [ ]:
wordlist <- c("SQL", "Python", ".NET", "noSQL", "sneakers", 'jij')
test_table <- word_vec[word_vec %in% tolower(wordlist)]
test_table
unique(test_table)
data.frame(x = toString(unique(test_table)))


## Get time since placement

In [30]:
library(stringr)
test_url = 'https://nl.indeed.com/viewjob?jk=33d4be78835757d1&tk=1ft2v35nf3b5m000&from=serp&vjs=3&advn=4610837443530944&adid=371890778&ad=-6NYlbfkN0A_cWh5ufxJCEatf9FOtgdwl-LajcKrnY3NwdklpeSDnjaH9EsBM6ngV0apa0_Kx2a5GKCW4uS0O7MdMAqsdKUS4q0L_C77bzQCZHbh2VakwHOoBSx_gXNPBJOeqxq7IUUc4s9CBf_lt5ktVCCKOckMPI8DQ-kKHnVfI97NdL-7of8Oo7OCSaCBZL8Z41xkIGi-jjU-emgJ8f1utRtU276rmh-6Rww5rOQKaWr9wwgGy6sYrcY4MEIcYBkbm5BychuYGL0AFbd7Vy65yEMtdbRprQHT9KeUwpjRNpt6FLJMpPGJcxEUzshqpQrMGS0B_CRCLlmxqTWqBEPfpy3Y7aujfdqxmlVIkXqhghqlEzA3Ag==&sjdu=0ZFwD5rbjMRcHz87Kzx_g4iztdTb_2bOBnSpyL0eJqrnoH0zhNHGeUS0d52A-25mtSPp3ZGAbkfQnEhseHJ_YqiTRf1RGyCMrfm3GKjpLJXmRb1As35_i94VtIfO7AZtgI0QXqDAYweserYyqDu1JUW2c6S_FT2kfNZmCatNflk'
test_url2 = 'https://nl.indeed.com/viewjob?jk=a4f0bcd239475ac5&q=data+analist&tk=1ftanb2kisu17800&from=web&advn=9087037695885843&adid=383523449&ad=-6NYlbfkN0D_XjDCAAdMmk6duX_ctb43y51tBCQMBYqAQgUo9Z3gKkSSAcm5qB6MEluEyQCzZ4Qvktb_X2s0ncEMl2dcKSdOuPOzHeSOf6ZwGIA7KGgoH4fSW1JFduihoiyO8TBp2nKKYBEHE7iAi4KAZ2m6AiqwzrmuN3eJma-6mwzCf030BjJnuXNyZrUgc66Jy0ljyxBBJwVnO20F0l0IQ2F42VdP4HMPZB643QzoQbUTllGRYOw9axf9q1WCiqw1DcpbA0wJ6mrqXnCm1SuydyEQ0eIbcOblRKZj8OKDQ29FSs8YWeKq7m-fOy7CSlIAtAQfaOZmkEV6nilpTrISaNlsI0anpaZOvRqojcG9fLWfubbWHopLAhwris69mgtY5tnBSvM%3D&pub=4a1b367933fd867b19b072952f68dceb&vjs=3'

time_since_placed_text <- read_html(test_url) %>%
    html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
    html_text()

# check if string does not contain integer using regex
if (!grepl("\\d", time_since_placed_text)){
    time_since_placed_text <- read_html(test_url2) %>%
        html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
        html_text()
}

time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]] %>%
    as.numeric()

time_since_placed


[1] 9

# Get placement date

In [60]:
library(lubridate)
time_since_placed
test_date <- today() - time_since_placed
class(test_date)
class(today())
class(time_since_placed)
class(today() - time_since_placed)
test <-today()
class(test)

[1] 15

[1] "Date"

[1] "Date"

[1] "numeric"

[1] "Date"

[1] "Date"

# Debugging

In [52]:
job_base_url <- 'https://nl.indeed.com'
job_base_search_url <- 'https://nl.indeed.com/jobs?q=data&start='
# url_job_search <- read_html('https://nl.indeed.com/jobs?q=data')

wordlist <- read.table(
    '..\\Capstone\\trimmed_skill_names.txt',
     header = TRUE
     ) %>% .$x

jobs_df <- data.frame(
    Company = character(),
    job_title = character(),
    salary = character(),
    skills = character(),
    days_online = numeric(),
    job_desc = character(),
    job_link = character()
)



# when url is longer than 200 chars it is a job url
# nchar(job_urls[[j]]) > 200
if (TRUE){
    full_job_url <- "https://nl.indeed.com/rc/clk?jk=e680b1eb61db2605&fccid=ba07516c418dda52&vjs=3"
    # read html
    job_html_read <- read_html(full_job_url)

    # extract company name
    company_name <- job_html_read %>%
        html_nodes(css= 'head > meta:nth-child(15)') %>%
        html_attr('content')


    # extract title
    job_title <- job_html_read %>%
        html_nodes('h1') %>%
        html_text()
    
    
    time_since_placed_text <- job_html_read %>%
        html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
        html_text()

    
    # check if string does not contain integer using regex
    if (!grepl("\\d", time_since_placed_text) & !grepl("Vandaag", time_since_placed_text)){
        time_since_placed_text <- job_html_read %>%
            html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
            html_text()
    }

    if (grepl("\\d", time_since_placed_text)){
        time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]]
    } else if (grepl("Vandaag", time_since_placed_text)) {
        time_since_placed <- 0
    } else {
       time_since_placed <- NA
    }


    # extract description
    job_description <- job_html_read %>%
        html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
        html_text()
    

    # remove non alpha numeric characters
    job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

    # generate wordvector
    wordvec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

    # extract skills from word vector
    skill_table <- wordvec[wordvec %in% tolower(wordlist)] %>%
        unique()

    # probeer salari op te halen via specifieke salary indicator
    salary_indicator <- job_html_read %>%
        html_nodes(css= '#salaryInfoAndJobType') %>%
        html_text()
    
  
    # Als de € niet voorkomt in het bestand probeer dan de salary distribution
    if (length(salary_indicator) < 1){
        salary_indicator <- job_html_read %>%
            html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
            html_text()
    }

    if (length(salary_indicator) < 1){
        salary <- NA
    } else { # anders zoek de eerste salaris indicatie
        salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
        salary <- substring(salary_indicator, salary_start_num, salary_start_num + 7) 
    }
    
    jobs_df[nrow(jobs_df) + 1,] = c(
        company_name,
        job_title,
        salary,
        toString(unique(skill_table)),
        as.numeric(time_since_placed),
        job_description,
        full_job_url)
    
    Sys.sleep(4)
}

[1] TRUE
[1] 36
[1] 15
[1] ""


In [57]:
substring(salary_indicator, salary_start_num, salary_start_num + 7)

[1] "\200 3.333 "